This is for generating Weather Data contraining
    1. External Air Temperature
    2. Global Horizontal Irradiance (GHI)
For running the dwelling heating simulation model

The source datafile is downloaded from ShinyWeatherData, which is a free service for downloading weather data.

In [13]:

import re
import pandas as pd

# --- path to the raw file you downloaded from ShinyWeatherData -------------
FILE = r"E:\Python projects\RC model\Weather Data\file14cff3fc3c1_lat=51.5_lng=-0.25_period=20180101-20231231.csv"          # ← change if your filename differs
OUT  = r"E:\Python projects\RC model\Weather Data\Weather_NW9_18_23.csv"              # optional: where to save the result
# ---------------------------------------------------------------------------

df = pd.read_csv(FILE, skiprows=11, usecols=['datetime','t2m','ssrd'],index_col='datetime')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
df.rename(columns={'t2m': "External_Air_Temperature", 'ssrd': "GHI"}, inplace=True)
df_30 = df.resample("30min").mean().interpolate(method="linear")
print(df_30.head(50))

df_30.to_csv(OUT, index=True, header=True,index_label='Timestamp')

                     External_Air_Temperature   GHI
datetime                                           
2018-01-01 00:00:00                      6.90   0.0
2018-01-01 00:30:00                      6.60   0.0
2018-01-01 01:00:00                      6.30   0.0
2018-01-01 01:30:00                      6.10   0.0
2018-01-01 02:00:00                      5.90   0.0
2018-01-01 02:30:00                      5.80   0.0
2018-01-01 03:00:00                      5.70   0.0
2018-01-01 03:30:00                      5.55   0.0
2018-01-01 04:00:00                      5.40   0.0
2018-01-01 04:30:00                      5.40   0.0
2018-01-01 05:00:00                      5.40   0.0
2018-01-01 05:30:00                      5.45   0.0
2018-01-01 06:00:00                      5.50   0.0
2018-01-01 06:30:00                      5.40   0.0
2018-01-01 07:00:00                      5.30   0.0
2018-01-01 07:30:00                      5.25   0.0
2018-01-01 08:00:00                      5.20   0.0
2018-01-01 0

This following part is for analysing and visualizing the metadata of dwellings that are selected to construct the LVN model.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
# --- path to the files ----------------------------------------------
FILE_META = r"E:\GitHubProjects\LV network\Codes\Data\EoH Dwellings Metadata.xlsx"   
# ← change if your filename differs
FILE_PARAMS = r"E:\GitHubProjects\LV network\Codes\Data\1R1C1P1S_filtered_filtered.csv"
# ---------------------------------------------------------------------------
df_meta = pd.read_excel(FILE_META, sheet_name=r'Sheet1', index_col='Property_ID')
df_params = pd.read_csv(FILE_PARAMS, index_col='dataset')

df_params.index = df_params.index.str[:7]
print(df_params.head())

            model winter_season          train_start            train_end  \
dataset                                                                     
EOH0003  1C1R1P1S     2021-2022  2021-12-03 00:00:00  2022-01-30 23:30:00   
EOH0014  1C1R1P1S     2021-2022  2021-12-23 00:00:00  2022-02-19 23:30:00   
EOH0018  1C1R1P1S     2021-2022  2021-12-04 00:00:00  2022-02-17 23:30:00   
EOH0021  1C1R1P1S     2021-2022  2021-12-02 00:00:00  2022-02-22 23:30:00   
EOH0025  1C1R1P1S     2021-2022  2021-12-16 00:00:00  2022-02-27 23:30:00   

                   val_start              val_end  \
dataset                                             
EOH0003  2021-12-14 00:00:00  2022-02-08 23:30:00   
EOH0014  2021-12-25 00:00:00  2022-02-28 23:30:00   
EOH0018  2021-12-07 00:00:00  2022-02-06 23:30:00   
EOH0021  2021-12-20 00:00:00  2022-02-18 23:30:00   
EOH0025  2022-01-12 00:00:00  2022-02-15 23:30:00   

                                                train_days  \
dataset                   

In [8]:
df_meta_filtered = df_meta[df_meta.index.isin(df_params.index)]
print(len(df_meta_filtered))
print(len(df_meta))
print(len(df_params))

177
283
508


In [9]:
df_meta_joint = df_meta.join(df_params, how='inner')
print(len(df_meta_joint))

177


In [11]:
df_meta_joint = df_meta_joint[df_meta_joint['House_Env']=='Urban']
print(len(df_meta_joint))


147


In [14]:
df_meta_joint['No_occupants'] = df_meta_joint['No_Adults'] + df_meta_joint['No_Child']
print(df_meta_joint['No_occupants'].value_counts())

No_occupants
2    54
3    32
4    32
1    22
5     7
Name: count, dtype: int64


In [12]:
df_meta_joint.to_csv(r"E:\GitHubProjects\LV network\Codes\Data\EoH_Dwellings_Metadata_filtered.csv", index=True, header=True,index_label='Property_ID')

This following part is for converting the NEcase weather data to a 30-min time resolution file containing only temperature and solar radiation data.

In [1]:
import pandas as pd

in_path = r"E:\GitHubProjects\LV network\Input data\NEcase_20_21.csv"                      # change if needed
out_path = r"E:\GitHubProjects\LV network\Input data\NEcase_20_21_t2m_ssrd_30min.csv"      # output file

# 1) Read (skip metadata lines starting with '#')
df = pd.read_csv(in_path, comment="#")

# 2) Pick timestamp column and keep only what you need
ts_col = "datetime" if "datetime" in df.columns else "datetime_lst"
df = df[[ts_col, "t2m", "ssrd"]].rename(columns={ts_col: "timestamp"})

# 3) Parse timestamp + ensure numeric
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df["t2m"] = pd.to_numeric(df["t2m"], errors="coerce")
df["ssrd"] = pd.to_numeric(df["ssrd"], errors="coerce")

df = df.dropna(subset=["timestamp"]).set_index("timestamp").sort_index()

# 4) Upsample to 30-min grid
df_30 = df.resample("30min").asfreq()

# 5) Fill rules:
#    - t2m: interpolate in time
#    - ssrd: duplicate (carry forward)
df_30["t2m"] = df_30["t2m"].interpolate(method="time")
df_30["ssrd"] = df_30["ssrd"].ffill()

# 6) Write out
df_30.reset_index().to_csv(out_path, index=False)
print(f"Wrote: {out_path}")


Wrote: E:\GitHubProjects\LV network\Input data\NEcase_20_21_t2m_ssrd_30min.csv


In [2]:
df = pd.read_csv(r"E:\GitHubProjects\LV network\Input data\Newcastle_Urban_Case_meta.csv")
df['No_occupants'] = df['No_Adults'] + df['No_Child']
df.to_csv(r"E:\GitHubProjects\LV network\Input data\Newcastle_Urban_Case_meta_updated.csv", index=False)